In [52]:
%pip install folium
import folium 
from folium.plugins import HeatMap
import json
import math
import pandas as pd 
import numpy as np
import requests

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:


url = "https://data.brisbane.qld.gov.au/api/explore/v2.1/catalog/datasets/traffic-data-at-intersection/exports/json?lang=en&timezone=Australia%2FBrisbane"

# Define the API key and the headers
api_key = "APIKEY"
headers = {
    "Authorization": f"Apikey {api_key}"
}

try:
    response = requests.get(url, headers=headers, verify=True)
    response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)

    data = response.json()
    
    # Convert JSON response to DataFrame
    PDTrafficVolume = pd.DataFrame(data)
  #  print(df.head())  # Print the first few rows of the DataFrame

except requests.exceptions.SSLError as ssl_err:
    print(f"SSL certificate verification failed: {ssl_err}")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")

In [ ]:
#open reference data. Using a local file as the data shouldn’t change much. can potentially change to make a web call to get most recent reference data.
RawReference = open('intersections-reference.json')

JSONReference = json.load(RawReference)

In [ ]:
#read the reference data into pandas
PDReference = pd.DataFrame(JSONReference)

#remove records with no latlon
non_missing = PDReference[['latitude', 'longitude']].notna()
df_clean = PDReference[non_missing['latitude'] & non_missing['longitude']]


In [ ]:
#Join the reference data with the live traffic data
merged_df = pd.merge(df_clean,PDTrafficVolume, how='left', left_on=['tsc'], right_on=['tsc'])

filtered_lat_df = merged_df[['latitude','longitude','tsc','lane','ds1','ds2','ds3','ds4']]

#remove rows where lane is None and ds1 is None
filtered_lat_df  = filtered_lat_df .dropna(subset=['lane', 'ds1'], how='all')

#filtered_lat_df['ds1'] = filtered_lat_df['ds1'].fillna(0)

# Convert to numeric type
filtered_lat_df['ds1'] = pd.to_numeric(filtered_lat_df['ds1'])
filtered_lat_df['ds2'] = pd.to_numeric(filtered_lat_df['ds2'])
filtered_lat_df['ds3'] = pd.to_numeric(filtered_lat_df['ds3'])
filtered_lat_df['ds4'] = pd.to_numeric(filtered_lat_df['ds4'])

def conditional_mean(series):
    if series.isna().all():
        return np.nan
    return series.mean()


group_means = filtered_lat_df.groupby('tsc').agg({
    'latitude': 'first',
    'longitude': 'first',
    'ds1': conditional_mean,
    'ds2': conditional_mean,
    'ds3': conditional_mean,
    'ds4': conditional_mean
}).reset_index()

#Get the mean degree of saturation excluding lanes that are not present on the road. 
group_means['ds_avg'] = group_means[['ds1', 'ds2', 'ds3', 'ds4']].mean(axis=1, skipna=True)
group_means = group_means.dropna(subset=['ds_avg'])


In [ ]:

#change the lane saturation to a decimal so it can be used to vary the heatmap colours. 
group_means['ds_avg'] = group_means['ds_avg'] * 0.01

# Ensure all values in group_means['ds1'] are changed to 1 if they are greater than 1
group_means['ds_avg'] = np.where(group_means['ds_avg'] > 1, 1, group_means['ds_avg'])


In [ ]:
#set up map
map_osm = folium.Map(location=[-27.467951, 153.020577],zoom_start = 16)

data = group_means[['latitude','longitude','ds_avg']]

HeatMap(data).add_to(map_osm)
map_osm